# 반복 단어, 문장 1개로 통일

In [73]:
!pip install rouge

  Obtaining dependency information for rouge from https://files.pythonhosted.org/packages/32/7c/650ae86f92460e9e8ef969cc5008b24798dcf56a9a8947d04c78f550b3f5/rouge-1.0.1-py3-none-any.whl.metadata


In [1]:
import pandas as pd
import json
import matplotlib.pyplot as plt
import seaborn as sns
import re
from copy import deepcopy
import warnings
warnings.filterwarnings("ignore", category=pd.errors.SettingWithCopyWarning)
import rouge

In [2]:
def make_dataframe(path: str) -> pd.DataFrame:
    """
    Read a json file and return a pandas DataFrame.

    Parameters:
    path (str): Path to the json file.

    Returns:
    pd.DataFrame: DataFrame of the json file.
    """
    # Read the json file
    with open(path, 'r') as file:
        data = json.load(file)

    # Create a DataFrame
    # columns = ['id', 'conversation', 'subject_keyword', 'output']
    df = pd.DataFrame(data)
    df['conversation'] = df['input'].apply(lambda x: x['conversation'])
    df['subject_keyword'] = df['input'].apply(lambda x: x['subject_keyword'])

    # Drop the 'input' column
    df.drop('input', axis=1, inplace=True)

    # Speakers in the conversation
    df['speakers'] = df['conversation'].apply(lambda turns: list(set(turn['speaker'] for turn in turns)))

    # Reorder the columns
    df = df[['id', 'conversation', 'subject_keyword', 'speakers', 'output']]

    return df

In [6]:
train_df = make_dataframe('../resource/data/일상대화요약_train.json')
dev_df = make_dataframe('../resource/data/일상대화요약_dev.json')
test_df = make_dataframe('../resource/data/일상대화요약_test.json')
filtered_train_df = make_dataframe('./train.json')
filtered_dev_df = make_dataframe('./dev.json')
filtered_test_df = make_dataframe('./test.json')

In [4]:
train_df

,id,conversation,subject_keyword,speakers,output
0,nikluge-2024-일상 대화 요약-train-000001,"[{'speaker': 'SD2000001', 'utterance': '저는 여행 ...",[해외여행],"[SD2000001, SD2000002]",이 대화에서 화자들은 좋았던 여행지와 기억나는 주요 명소에 대해 이야기했습니다. S...
1,nikluge-2024-일상 대화 요약-train-000002,"[{'speaker': 'SD2000002', 'utterance': '여행 다닐 ...","[여행 스타일, 숙소, 음식]","[SD2000001, SD2000002]","이 대화에서 화자들은 쇼핑하기, 숙소, 음식 등 각자 선호하는 여행 스타일에 대해 ..."
2,nikluge-2024-일상 대화 요약-train-000003,"[{'speaker': 'SD2000001', 'utterance': '어~ 그래서...","[여행지, 일화]","[SD2000001, SD2000002]",이 대화에서 화자들은 기억에 남는 여행지와 일화에 대해 이야기했습니다. SD2000...
3,nikluge-2024-일상 대화 요약-train-000004,"[{'speaker': 'SD2000005', 'utterance': 'name1 ...","[음식, 선호, 길거리 음식, 싫어하는 음식]","[SD2000006, SD2000005]",두 화자는 이 대화에서 좋아하는 음식과 싫어하는 음식에 대해 말했습니다. SD200...
4,nikluge-2024-일상 대화 요약-train-000005,"[{'speaker': 'SD2000006', 'utterance': '그러면 na...","[다이어트 식품, 마실것]","[SD2000006, SD2000005]",두 화자는 이 대화에서 다이어트 식품과 좋아하는 음료에 대해 이야기했습니다. SD2...
...,...,...,...,...,...
501,nikluge-2024-일상 대화 요약-train-000502,"[{'speaker': 'SD2100596', 'utterance': '그 comp...",[company-name5],"[SD2100596, SD2100595]",두 화자는 이 대화에서 유명 유튜버의 식당 추천과 음식 장사의 성공 요인에 대해서 ...
502,nikluge-2024-일상 대화 요약-train-000503,"[{'speaker': 'SD2100598', 'utterance': '나 이번 달...",[행복 주택],"[SD2100597, SD2100598]",이 대화에서 화자들은 주택 마련 방법과 고충에 대해서 이야기했습니다. SD21005...
503,nikluge-2024-일상 대화 요약-train-000504,"[{'speaker': 'SD2100598', 'utterance': '저는 지금 ...",[수익],"[SD2100597, SD2100598]",이 대화에서 화자들은 현재 소득에서 수익 창출 방법에 대해 이야기 했습니다. SD2...
504,nikluge-2024-일상 대화 요약-train-000505,"[{'speaker': 'SD2100599', 'utterance': '동네에 맛있...",[점심],"[SD2100600, SD2100599]",이 대화에서 화자들은 좋아하는 치킨 브랜드와 어제 저녁 먹은 음식과 점심으로 먹은 ...


In [5]:
# utterance 내에 '\b([가-힣a-zA-Z0-9_]+)\s+\1\b'를 만족하는 부분이 있는지 확인하고 있다면 set에 추가하고 마지막에 return하는 함수

def find_repeated_words(df: pd.DataFrame, pattern) -> set:
    """
    Find repeated words in the utterances of the DataFrame.

    Parameters:
    df (pd.DataFrame): DataFrame of the conversations.

    Returns:
    set: Set of repeated words.
    """
    # Set to store the repeated words
    repeated_words = set()
    count = 0

    # Iterate over the DataFrame
    for idx, row in df.iterrows():
        # Extract the conversation
        conversation = row['conversation']

        # Iterate over the turns
        for turn in conversation:
            # Extract the utterance
            utterance = turn['utterance']

            # Find all repeated words
            matches = re.findall(pattern, utterance)

            # Add the repeated words to the set
            repeated_words.update(matches)

            count+=len(matches)

    print(f"Total number of repeated words found: {len(repeated_words)}")
    print(f"Total number of repeated words found: {count}")

    return repeated_words

In [43]:
a= "우리는 청팀 백팀이었거든? 그래가지고 그냥 1반 1반 1반 청팀 2반 백팀 해서 그냥 운동회 체육 대회라기보다 운동회 느낌으로 했어."
prev = re.sub(r'\b([가-힣a-zA-Z0-9_]+)\s+\1\b', r'\1', a)
post = re.sub(r'\b(\w+)\b(?:\s+\1\b)+', r'\1', a)
print('prev :',prev)
print('post :',post)

prev : 우리는 청팀 백팀이었거든? 그래가지고 그냥 1반 1반 청팀 2반 백팀 해서 그냥 운동회 체육 대회라기보다 운동회 느낌으로 했어.
post : 우리는 청팀 백팀이었거든? 그래가지고 그냥 1반 청팀 2반 백팀 해서 그냥 운동회 체육 대회라기보다 운동회 느낌으로 했어.


In [29]:
repeated_words2 = find_repeated_words(train_df, r'\b(\w+)\s+\1\b')

Total number of repeated words found: 567
Total number of repeated words found: 1224


In [17]:
repeated_words = find_repeated_words(dev_df, r'\b(\w+)\b(?:\s+\1\b)+')

Total number of repeated words found: 121
Total number of repeated words found: 163


In [39]:
repeated_words = find_repeated_words(train_df, r'\b([가-힣a-zA-Z0-9_]+)\s+\1\b')

Total number of repeated words found: 567
Total number of repeated words found: 1224


In [10]:
repeated_words = find_repeated_words(filtered_train_df, r'\b([가-힣a-zA-Z0-9_]+)\s+\1\b')
repeated_words

Total number of repeated words found: 27
Total number of repeated words found: 28


{'1반',
 '600그램에',
 '걱정이',
 '그냥',
 '그런',
 '돌고',
 '둘',
 '메꿔주고',
 '세',
 '아니',
 '아니야',
 '야',
 '약간',
 '예',
 '오아후',
 '이거',
 '이런',
 '이렇게',
 '자기',
 '자꾸',
 '정말',
 '조금',
 '주',
 '풍미',
 '한',
 '헉',
 '홍대'}

In [11]:
repeated_words = find_repeated_words(filtered_dev_df, r'\b([가-힣a-zA-Z0-9_]+)\s+\1\b')
repeated_words

Total number of repeated words found: 4
Total number of repeated words found: 4


{'그냥', '되게', '쪼금씩', '툭'}

In [12]:
repeated_words = find_repeated_words(filtered_test_df, r'\b([가-힣a-zA-Z0-9_]+)\s+\1\b')
repeated_words

Total number of repeated words found: 15
Total number of repeated words found: 17


{'골',
 '그런',
 '되게',
 '바라고',
 '아이',
 '어디',
 '여기',
 '잠',
 '제가',
 '종이',
 '진짜',
 '최고',
 '툭',
 '한',
 '항상'}

In [4]:
a = "어~ 슈퍼맨이 돌아왔다랑 아빠 어디가 왜 좋아해?"
b = "왜 좋아해?"

In [5]:
b in a

True

In [134]:
rouge_score = rouge.Rouge().get_scores

def find_repeated_sentence(df: pd.DataFrame) -> set:
    """
    Find repeated sentences in the DataFrame.

    Parameters:
    df (pd.DataFrame): DataFrame of the conversations.

    Returns:
    set: Set of repeated sentences.
    """
    # Set to store the repeated sentences
    repeated_sentences = set()
    count = 0

    # Iterate over the DataFrame
    for idx, row in df.iterrows():
        # Extract the conversation
        conversation = row['conversation']

        # Iterate over the turns
        for i, turn in enumerate(conversation):
            # Extract the utterance
            utterance = turn['utterance']

            if i != 0:
                prev_speaker = conversation[i-1]['speaker']
                prev_utterance = conversation[i-1]['utterance']

                utterance, prev_utterance = utterance.split('.')[0].strip(), prev_utterance.split('.')[-1].strip()
                if prev_speaker == turn['speaker']:
                    # Check if last sentence of the previous utterance is the same as first sentence of the current utterance
                    # if utterance.split('.')[0] in prev_utterance.split('.')[-1]:
                    # if utterance in prev_utterance:
                    if prev_utterance in (".","") or utterance in (".",""):
                        continue
                    else:
                        if rouge_score([utterance], [prev_utterance])[0]['rouge-1']['f'] > 0.5:
                            # Add the repeated sentences to the set
                            repeated_sentences.add(utterance)
                            count+=1
                            
    print(f"Total number of repeated sentences found: {len(repeated_sentences)}, frequency: {count}")

    return repeated_sentences

In [136]:
repeated_sentences = find_repeated_sentence(train_df)

Total number of repeated sentences found: 5, frequency: 5


In [137]:
repeated_sentences

{'그~ 뭐지',
 '그러니까 뭐 이~ 스포츠 선수들 중에 가장 좋아했던 사람',
 '뭐 화장품 산업을 발달시킨다든지 이런 부분은 또 장점이 될 수도 있지만 저는 사 근데 또 개인적으로는 어~ 쪼끔 그 정신',
 '안 먹으면 이제는 어~ 힘이 없으니까 말이 안 나올 거 같고 그냥 적게 먹는 먹는 게 다이어트 하는 방법인 거 같아요',
 '앞으로 어~ 먹 먹어 보고 싶은 먹거리가 있나요?'}

In [61]:
repeated_sentences

{'',
 '가',
 '그',
 '아직 열고 있긴 한데 건너편에 재개발이 되다 보니까 상권들이 다 안 좋아져서 많이들 맛집들이 문을 닫으려고 하는 거 같아.',
 '우리도 이사를 해야 됐어서 결국에는 엄지 손가락 길이었던 거북이를 한 손바닥 크기만큼 키운 적이 있었는데 얘네들을 어떻게 처리할 수가 없는 상황이었어.'}

In [79]:
rouge.Rouge().get_scores(' '.join(repeated_sentences), ' '.join(repeated_sentences))

[{'rouge-1': {'r': 1.0, 'p': 1.0, 'f': 0.999999995},
  'rouge-2': {'r': 1.0, 'p': 1.0, 'f': 0.999999995},
  'rouge-l': {'r': 1.0, 'p': 1.0, 'f': 0.999999995}}]